In [3]:
import os
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_groq import ChatGroq
import tempfile

In [4]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

client = ChatGroq(
    temperature=0.5,
    model_name="llama3-8b-8192",
    groq_api_key=groq_api_key
)

In [5]:
def load_document(file_path):
    """Load a document and return the text."""
    extension = os.path.splitext(file_path)[1].lower()
    if extension == ".pdf":
        return PyPDFLoader(file_path).load()
    elif extension == ".docx" or extension == ".doc":
        return Docx2txtLoader(file_path).load()
    elif extension == ".txt":
        return TextLoader(file_path).load()
    else:
        raise ValueError(f"Unsupported file extension: {extension}")

In [6]:
sample_pdf_path = "temp.pdf"
document_text = load_document(sample_pdf_path)

In [7]:
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=768, chunk_overlap=128)
text_chunks = text_splitter.split_documents(document_text)

In [8]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(documents=text_chunks, embedding=embedding_model)


/var/folders/vh/44rrbdlx5vxgbtcv7fhxgd0c0000gn/T/ipykernel_24729/669638979.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/moka/Desktop/TARS/myenv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/moka/Desktop/TARS/myenv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=client, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}), 
    memory=memory
)

In [10]:
def ask_question(question, chain, history):
    result = chain({"question": question, "chat_history": history})
    history.append((question, result["answer"]))
    return result["answer"]

In [11]:
example_queries = [
    "What is the document about?",
    "Summarize the key points from the first section.",
    "Can you explain how the method works?",
    "What are the results mentioned in the document?",
]


chat_history = []


for query in example_queries:
    response = ask_question(query, conversational_chain, chat_history)
    print(f"Q: {query}\nA: {response}\n")

/var/folders/vh/44rrbdlx5vxgbtcv7fhxgd0c0000gn/T/ipykernel_24729/2412595810.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = chain({"question": question, "chat_history": history})


Q: What is the document about?
A: This document appears to be a collection of references to academic papers in the field of natural language processing (NLP) and machine learning. The references are listed in a specific format, with each reference including the authors, title, publication information, and a unique identifier (e.g. [1], [2], etc.).

The papers referenced appear to be related to various topics in NLP, including parsing, machine translation, summarization, and neural networks. The references may be used as a bibliography or a list of sources for a research paper or article on NLP.

Q: Summarize the key points from the first section.
A: The main topics discussed in the document appear to be:

1. The approach taken in the model, which involves self-attention.
2. The potential benefits of self-attention, including more interpretable models.
3. The training regime for the models, including the training data and batching.

It seems that the document is discussing the architect